In [1]:
from sqlalchemy import extract, desc
import datetime
import time
import MetaTrader5 as mt5
import pandas as pd

import options_estrategies as opt_est
from options_estrategies import Option_Due, InformationType
import models as model

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

estrategies = opt_est.options_estrategies(model, model.PETR4, model.PETR4_OPTIONS)

In [3]:
def convert_dict_from_update_ticks_to_dataframe(tick_data):
    df_tick_data = pd.DataFrame.from_dict(tick_data)
    df_tick_data = df_tick_data.drop(columns=['_sa_instance_state','timestamp_option','days_to_due_date'])
    df_tick_data['ratio'] = df_tick_data['last_tick'] - df_tick_data['last_tick'].shift(1)
    df_tick_data = df_tick_data[['option_name', 'strike', 'bid', 'ask', 'last_tick','ratio','stock_price','deal_type_zone','updated_at']]      
    return df_tick_data


In [4]:
updated_ticks = estrategies.update_quotes(just_last_update=False,mode=InformationType.Real_Time)
ticks_to_process = updated_ticks[0]

options = []
for i in ticks_to_process:
    options.append(i.__dict__)

dataframe_historical_ticks = convert_dict_from_update_ticks_to_dataframe(options)
dataframe_historical_ticks
ratio_statistic_finder = dataframe_historical_ticks[['updated_at','option_name','ratio']]
ratio_statistic_finder = ratio_statistic_finder.set_index('updated_at')
ratio_statistic_finder = ratio_statistic_finder.between_time('10:10', '17:00')
ratio_statistic_finder = ratio_statistic_finder.reset_index()
ratio_statistic_finder = ratio_statistic_finder.pivot(index='updated_at', columns='option_name', values='ratio')
ratio_statistic_finder = ratio_statistic_finder.dropna(axis=1)
ratio_statistic_finder

ratio_statistic_data = {}
for (column, value) in ratio_statistic_finder.iteritems():
    ratio_statistic_data.update({ column: { "Mean": ratio_statistic_finder[column].mean(), "StdDev": ratio_statistic_finder[column].std(), "2xStdDev": (ratio_statistic_finder[column].std()*2)  }}) 

ratio_statistic_dataframe = pd.DataFrame.from_dict(ratio_statistic_data)
ratio_statistic_dataframe = ratio_statistic_dataframe.transpose()

updated_ticks = estrategies.update_quotes(just_last_update=True,mode=InformationType.Real_Time)
ticks_to_process = updated_ticks[0]
options_updated = []
for i in ticks_to_process:
    options_updated.append(i.__dict__)

df_options_updated = convert_dict_from_update_ticks_to_dataframe(options_updated)
df_options_updated.reset_index()
df_options_updated = df_options_updated.set_index('option_name')
df_options_updated = pd.merge(df_options_updated, ratio_statistic_dataframe, left_index=True, right_index=True)

df_options_updated['above_mean'] = abs(df_options_updated['ratio']) > abs(df_options_updated['Mean'])
df_options_updated['above_2x_std_dev'] = abs(df_options_updated['ratio']) > abs(abs(df_options_updated['Mean']) + df_options_updated['2xStdDev'])
df_options_updated['a2xStdDev_Price'] = abs(abs(df_options_updated['Mean']) + df_options_updated['2xStdDev'])
df_options_updated = df_options_updated.drop(columns=['updated_at','StdDev','2xStdDev'])
df_options_updated

,strike,bid,ask,last_tick,ratio,stock_price,deal_type_zone,Mean,above_mean,above_2x_std_dev,a2xStdDev_Price
PETRH269,26.21,1.83,1.85,1.83,-0.07,27.29,ITM,0.900899,False,False,3.003059
PETRH254,26.46,1.65,1.68,1.69,-0.14,27.29,ITM,-0.177657,False,False,0.239024
PETRH261,27.21,1.21,1.23,1.22,-0.16,27.29,ATM,-0.155286,True,False,0.206466
PETRH247,27.96,0.85,0.87,0.84,-0.12,27.29,OTM,-0.128170,False,False,0.177928
PETRH289,28.21,0.75,0.76,0.75,-0.09,27.29,OTM,-0.109376,False,False,0.139892
PETRH279,28.46,0.66,0.67,0.67,-0.08,27.29,OTM,-0.106606,False,False,0.134867
PETRH294,28.71,0.58,0.59,0.58,-0.09,27.29,OTM,-0.094483,False,False,0.124936
PETRH243,28.96,0.50,0.52,0.51,-0.07,27.29,OTM,-0.086447,False,False,0.113897
PETRH299,29.21,0.44,0.45,0.43,-0.08,27.29,OTM,-0.081365,False,False,0.111006


In [8]:
returned_ratio_compare = estrategies.ratio_between_strikes_statistic_realtime_compare()
returned_ratio_compare


,strike,bid,ask,last_tick,ratio,stock_price,deal_type_zone,Mean,above_mean,above_2x_std_dev,a2xStdDev_Price
PETRH269,26.21,1.81,1.83,1.82,-0.36,27.28,ITM,0.844487,False,False,3.018486
PETRH254,26.46,1.63,1.67,1.69,-0.20,27.28,ITM,-0.222018,False,False,0.278513
PETRH261,27.21,1.20,1.22,1.22,-0.17,27.28,ATM,-0.175847,False,False,0.238022
PETRH247,27.96,0.84,0.86,0.85,-0.13,27.28,OTM,-0.141498,False,False,0.194798
PETRH289,28.21,0.75,0.76,0.76,-0.11,27.28,OTM,-0.125950,False,False,0.156980
PETRH279,28.46,0.66,0.67,0.66,-0.10,27.28,OTM,-0.118939,False,False,0.145217
PETRH294,28.71,0.57,0.59,0.58,-0.10,27.28,OTM,-0.110300,False,False,0.139599
PETRH243,28.96,0.50,0.51,0.50,-0.09,27.28,OTM,-0.101952,False,False,0.127504
PETRH299,29.21,0.43,0.45,0.44,-0.08,27.28,OTM,-0.093196,False,False,0.119326


In [17]:
returned_ratio_compare['above_2x_std_dev']['PETRH269'] = True
returned_ratio_compare['above_2x_std_dev']['PETRH261'] = True
returned_ratio_compare

<ipython-input-17-db36607e78d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returned_ratio_compare['above_2x_std_dev']['PETRH269'] = True
<ipython-input-17-db36607e78d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returned_ratio_compare['above_2x_std_dev']['PETRH261'] = True


,strike,bid,ask,last_tick,ratio,stock_price,deal_type_zone,Mean,above_mean,above_2x_std_dev,a2xStdDev_Price
PETRH269,26.21,1.81,1.83,1.82,-0.36,27.28,ITM,0.844487,False,True,3.018486
PETRH254,26.46,1.63,1.67,1.69,-0.20,27.28,ITM,-0.222018,False,False,0.278513
PETRH261,27.21,1.20,1.22,1.22,-0.17,27.28,ATM,-0.175847,False,True,0.238022
PETRH247,27.96,0.84,0.86,0.85,-0.13,27.28,OTM,-0.141498,False,False,0.194798
PETRH289,28.21,0.75,0.76,0.76,-0.11,27.28,OTM,-0.125950,False,False,0.156980
PETRH279,28.46,0.66,0.67,0.66,-0.10,27.28,OTM,-0.118939,False,False,0.145217
PETRH294,28.71,0.57,0.59,0.58,-0.10,27.28,OTM,-0.110300,False,False,0.139599
PETRH243,28.96,0.50,0.51,0.50,-0.09,27.28,OTM,-0.101952,False,False,0.127504
PETRH299,29.21,0.43,0.45,0.44,-0.08,27.28,OTM,-0.093196,False,False,0.119326


In [37]:
returned_ratio_compare = estrategies.ratio_between_strikes_statistic_realtime_compare()
returned_ratio = returned_ratio_compare.copy()
returned_ratio = returned_ratio.reset_index()
ratio_deviation_high = returned_ratio[returned_ratio['above_2x_std_dev'] == True]

return_operation = []

for index, row in ratio_deviation_high.iterrows():
    if index > 0:
        data_ratio = returned_ratio[index-1:index+ 1].copy()
        data_ratio.insert(0, 'operation', ['C','V'])
        return_operation.append(data_ratio)

return_operation

[]